In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, date_format, date_add
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, parent_dir)

from apis.airportdb import AirportDB
from apis.weatherbit import WeatherBitDB

spark = SparkSession.builder.appName("EDA with PySpark").getOrCreate()

24/09/03 00:15:25 WARN Utils: Your hostname, victor-gabriel-pc resolves to a loopback address: 127.0.1.1; using 192.168.31.210 instead (on interface wlp2s0)
24/09/03 00:15:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/03 00:15:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
os.getcwd()

'/home/victor-gabriel/Documents/repos/picpay-ml-eng/notebook'

In [3]:
df = spark.read.csv("../data/airports-database.csv", header=True, inferSchema=True)

In [4]:
df.show(5)

+---+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+--------------------+
| id|year|month|day|dep_time|sched_dep_time|dep_delay|arr_time|sched_arr_time|arr_delay|carrier|flight|tailnum|origin|dest|air_time|distance|hour|minute|          time_hour|                name|
+---+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+--------------------+
|  0|2013|    1|  1|   517.0|           515|      2.0|   830.0|           819|     11.0|     UA|  1545| N14228|   EWR| IAH|   227.0|    1400|   5|    15|2013-01-01 05:00:00|United Air Lines ...|
|  1|2013|    1|  1|   533.0|           529|      4.0|   850.0|           830|     20.0|     UA|  1714| N24211|   LGA| IAH|   227.0|    1416|   5|    29|2013-01-01 05:00:00|United Air Lines ...|
|  2|2013|    1|  1|   54

In [5]:
#get size of the data
print((df.count(), len(df.columns)))

(336776, 21)


In [6]:
#get flight unique values 
df.select("flight").distinct().count()

3844

In [7]:
df[df["flight"] == 1959].show(5)

+----+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+--------------------+
|  id|year|month|day|dep_time|sched_dep_time|dep_delay|arr_time|sched_arr_time|arr_delay|carrier|flight|tailnum|origin|dest|air_time|distance|hour|minute|          time_hour|                name|
+----+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+--------------------+
| 110|2013|    1|  1|   804.0|           810|     -6.0|  1103.0|          1116|    -13.0|     DL|  1959| N947DL|   JFK| MCO|   147.0|     944|   8|    10|2013-01-01 08:00:00|Delta Air Lines Inc.|
| 991|2013|    1|  2|   800.0|           810|    -10.0|  1102.0|          1116|    -14.0|     DL|  1959| N995DL|   JFK| MCO|   143.0|     944|   8|    10|2013-01-02 08:00:00|Delta Air Lines Inc.|
|1935|2013|    1|  3

In [8]:
df.groupBy("flight").count().show(5)

+------+-----+
|flight|count|
+------+-----+
|   496|   10|
|  1959|  330|
|  4935|   32|
|   471|  129|
|  1580|   92|
+------+-----+
only showing top 5 rows



Temos 336776 registros de voos. <br/>
Temos 3844 registros único de voos, ou seja, 3844 voos que foram feitos mais de uma vez e com diferentes aeronaves, atrasos, número da cauda, etc. <br/>

In [9]:
#filter df dep_time = null and arr_time = 
df.filter(df["dep_time"].isNull() & df["arr_time"].isNull()).count() 

8255

8255 registros de voos foram cancelados. <br/>

In [10]:
df.select("dep_delay").describe().show()

+-------+------------------+
|summary|         dep_delay|
+-------+------------------+
|  count|            328521|
|   mean|12.639070257304708|
| stddev| 40.21006089212997|
|    min|             -43.0|
|    max|            1301.0|
+-------+------------------+



In [11]:
not_cancelled_flights = df.filter(df["dep_time"].isNotNull() | df["arr_time"].isNotNull())

In [12]:
not_cancelled_flights.groupBy("dest").count().orderBy("count", ascending=False).show(5)

+----+-----+
|dest|count|
+----+-----+
| ATL|16898|
| ORD|16642|
| LAX|16076|
| BOS|15049|
| MCO|13982|
+----+-----+
only showing top 5 rows



In [13]:
df.groupBy(["origin", "dest"]).count().orderBy("count", ascending=False).show(5)

+------+----+-----+
|origin|dest|count|
+------+----+-----+
|   JFK| LAX|11262|
|   LGA| ATL|10263|
|   LGA| ORD| 8857|
|   JFK| SFO| 8204|
|   LGA| CLT| 6168|
+------+----+-----+
only showing top 5 rows



Maior tempo médio de atraso

In [14]:
df.groupBy("dest") \
    .mean("dep_delay") \
    .withColumnRenamed("avg(dep_delay)", "avg_dep_delay") \
    .orderBy("avg_dep_delay", ascending=False) \
    .show(5)

+----+------------------+
|dest|     avg_dep_delay|
+----+------------------+
| CAE|35.570093457943926|
| TUL| 34.90635451505017|
| OKC|30.568807339449542|
| BHM| 29.69485294117647|
| TYS|28.493955094991364|
+----+------------------+
only showing top 5 rows



Dia da semana com maior número de voos

In [15]:
df.select(['time_hour']).show()
df_with_weekday = df.withColumn(
    "day_of_week",
    date_format(col("time_hour"), "EEEE")
)

+-------------------+
|          time_hour|
+-------------------+
|2013-01-01 05:00:00|
|2013-01-01 05:00:00|
|2013-01-01 05:00:00|
|2013-01-01 05:00:00|
|2013-01-01 06:00:00|
|2013-01-01 05:00:00|
|2013-01-01 06:00:00|
|2013-01-01 06:00:00|
|2013-01-01 06:00:00|
|2013-01-01 06:00:00|
|2013-01-01 06:00:00|
|2013-01-01 06:00:00|
|2013-01-01 06:00:00|
|2013-01-01 06:00:00|
|2013-01-01 06:00:00|
|2013-01-01 05:00:00|
|2013-01-01 06:00:00|
|2013-01-01 06:00:00|
|2013-01-01 06:00:00|
|2013-01-01 06:00:00|
+-------------------+
only showing top 20 rows



In [16]:
df_with_weekday.select(["time_hour", "day_of_week"]).show(5)

+-------------------+-----------+
|          time_hour|day_of_week|
+-------------------+-----------+
|2013-01-01 05:00:00|    Tuesday|
|2013-01-01 05:00:00|    Tuesday|
|2013-01-01 05:00:00|    Tuesday|
|2013-01-01 05:00:00|    Tuesday|
|2013-01-01 06:00:00|    Tuesday|
+-------------------+-----------+
only showing top 5 rows



In [17]:
df_with_weekday.groupBy("day_of_week").count().orderBy("count", ascending=False).show()

+-----------+-----+
|day_of_week|count|
+-----------+-----+
|     Monday|50690|
|    Tuesday|50422|
|     Friday|50308|
|   Thursday|50219|
|  Wednesday|50060|
|     Sunday|46357|
|   Saturday|38720|
+-----------+-----+



Data Split for Model Training

In [18]:
df_with_weekday.show(2)

+---+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+--------------------+-----------+
| id|year|month|day|dep_time|sched_dep_time|dep_delay|arr_time|sched_arr_time|arr_delay|carrier|flight|tailnum|origin|dest|air_time|distance|hour|minute|          time_hour|                name|day_of_week|
+---+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+--------------------+-----------+
|  0|2013|    1|  1|   517.0|           515|      2.0|   830.0|           819|     11.0|     UA|  1545| N14228|   EWR| IAH|   227.0|    1400|   5|    15|2013-01-01 05:00:00|United Air Lines ...|    Tuesday|
|  1|2013|    1|  1|   533.0|           529|      4.0|   850.0|           830|     20.0|     UA|  1714| N24211|   LGA| IAH|   227.0|    1416|   5|    29|2013-01-01 05:00:00

In [19]:
X_columns = ["distance", "dep_delay", "arr_delay"]
y_columns = "air_time"

In [20]:
X = df.select(X_columns)
y = df.select(y_columns)

In [21]:
y.show()

+--------+
|air_time|
+--------+
|   227.0|
|   227.0|
|   160.0|
|   183.0|
|   116.0|
|   150.0|
|   158.0|
|    53.0|
|   140.0|
|   138.0|
|   149.0|
|   158.0|
|   345.0|
|   361.0|
|   257.0|
|    44.0|
|   337.0|
|   152.0|
|   134.0|
|   147.0|
+--------+
only showing top 20 rows



In [22]:
import requests
weatherbit_key = "1f22b701b6904b66a169a5f52eddbed4"
latitude = 40.6925
longitude = -74.1687
start_date = '2013-01-01'
end_date = '2013-01-02' # Sempre adicione +1 dia em relação ao start_date
url = 'https://api.weatherbit.io/v2.0/history/daily'
params = {
'lat': latitude,
'lon': longitude,
'start_date': start_date,
'end_date': end_date,
'key': weatherbit_key,
}
headers = {
'Accept': 'application/json',
}
response = requests.get(url, params=params, headers=headers)
data = response.json()
data


{'city_id': '5101798',
 'city_name': 'Newark',
 'country_code': 'US',
 'data': [{'clouds': 93,
   'datetime': '2013-01-01',
   'dewpt': -5,
   'dhi': 27,
   'dni': 230,
   'ghi': 99,
   'max_dhi': 92,
   'max_dni': 773,
   'max_ghi': 428,
   'max_temp': 5,
   'max_temp_ts': 1357056000,
   'max_uv': 0.7,
   'max_wind_dir': 279,
   'max_wind_spd': 8.2,
   'max_wind_spd_ts': 1357095600,
   'min_temp': -2.2,
   'min_temp_ts': 1357099200,
   'precip': 0,
   'precip_gpm': 0,
   'pres': 1011,
   'revision_status': 'final',
   'rh': 56,
   'slp': 1012,
   'snow': 0,
   'snow_depth': 18,
   'solar_rad': 25,
   't_dhi': 639,
   't_dni': 5513,
   't_ghi': 2380,
   't_solar_rad': 601,
   'temp': 2.8,
   'ts': 1357016400,
   'wind_dir': 279,
   'wind_gust_spd': 11.7,
   'wind_spd': 5.8}],
 'lat': 40.6925,
 'lon': -74.1687,
 'sources': ['997272-99999',
  '725020-14734',
  'imerg',
  'era5',
  'modis',
  'snodas'],
 'state_code': 'NJ',
 'station_id': '725020-14734',
 'timezone': 'America/New_York'}

In [23]:
data['data'][0]['wind_spd']

5.8

In [24]:
origins = df.select('origin').distinct()
dests = df.select('dest').distinct()
dests = dests.withColumnRenamed('dest', 'origin')
unique_locations = origins.union(dests).distinct()
location_list = [row['origin'] for row in unique_locations.collect()]

In [25]:
location_dict = AirportDB.get_airport_data(location_list)

In [26]:
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, FloatType

def get_location(airport_code):
    return location_dict.get(airport_code, None)

get_location_udf = udf(get_location,  ArrayType(FloatType()))

df_with_coordinates = df.withColumn("origin_coordinates", get_location_udf(df["origin"]))
df_with_coordinates.show(truncate=False)

+---+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+------------------------+-------------------+
|id |year|month|day|dep_time|sched_dep_time|dep_delay|arr_time|sched_arr_time|arr_delay|carrier|flight|tailnum|origin|dest|air_time|distance|hour|minute|time_hour          |name                    |origin_coordinates |
+---+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+------------------------+-------------------+
|0  |2013|1    |1  |517.0   |515           |2.0      |830.0   |819           |11.0     |UA     |1545  |N14228 |EWR   |IAH |227.0   |1400    |5   |15    |2013-01-01 05:00:00|United Air Lines Inc.   |[40.6925, -74.1687]|
|1  |2013|1    |1  |533.0   |529           |4.0      |850.0   |830           |20.0     |UA     |1714  |N24211 |LGA   |IAH |2

In [27]:
df_weather = df_with_coordinates.withColumn(
    "start_date", date_format(col("time_hour"), "yyyy-MM-dd")
).withColumn(
    "end_date", date_format(date_add(col("time_hour"),1), "yyyy-MM-dd")  #1 day after start_date
).select(
    col("origin_coordinates").alias("coordinates"),
    col("start_date"),
    col("end_date")
)

data_to_send = df_weather.rdd.map(lambda row: (
    row["coordinates"][0],  # latitude
    row["coordinates"][1],  # longitude
    row["start_date"],
    row["end_date"]
)).collect()

In [28]:
data_to_send[:5]

[(40.692501068115234, -74.168701171875, '2013-01-01', '2013-01-02'),
 (40.777198791503906, -73.87259674072266, '2013-01-01', '2013-01-02'),
 (40.639801025390625, -73.77890014648438, '2013-01-01', '2013-01-02'),
 (40.639801025390625, -73.77890014648438, '2013-01-01', '2013-01-02'),
 (40.777198791503906, -73.87259674072266, '2013-01-01', '2013-01-02')]

In [29]:
weather_data = WeatherBitDB.get_weather_list(data_to_send)

KeyboardInterrupt: 

In [ ]:
from pyspark.sql import Row

weather_rows = [
    Row(
        lat=lat,
        lon=lon,
        start_date=start_date,
        end_date=end_date,
        wind_spd=wind_spd
    )
    for (lat, lon, start_date, end_date), wind_spd in weather_data.items()
]

# Criar um DataFrame a partir da lista de Rows
df_weather = spark.createDataFrame(weather_rows)

In [ ]:
df_weather.show()

+------------------+------------------+----------+----------+--------+
|               lat|               lon|start_date|  end_date|wind_spd|
+------------------+------------------+----------+----------+--------+
|40.692501068115234|  -74.168701171875|2013-01-01|2013-01-02|     5.8|
|40.777198791503906|-73.87259674072266|2013-01-01|2013-01-02|     6.9|
|40.639801025390625|-73.77890014648438|2013-01-01|2013-01-02|     6.8|
+------------------+------------------+----------+----------+--------+



In [ ]:

df_with_coords_alias = df_with_coordinates.alias("df_coords")
df_weather_alias = df_weather.alias("df_weather")

# Realizando a junção com os alias
df_final = df_with_coords_alias.join(
    df_weather_alias,
    (col("df_coords.origin_coordinates")[0] == col("df_weather.lat")) &
    (col("df_coords.origin_coordinates")[1] == col("df_weather.lon")) &
    (col("df_coords.start_date") == col("df_weather.start_date")) &
    (col("df_coords.end_date") == col("df_weather.end_date")),
    "left"
)

In [ ]:
df_final.show()

24/09/02 22:30:53 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+--------------------+-------------------+----------+----------+------------------+------------------+----------+----------+--------+
| id|year|month|day|dep_time|sched_dep_time|dep_delay|arr_time|sched_arr_time|arr_delay|carrier|flight|tailnum|origin|dest|air_time|distance|hour|minute|          time_hour|                name| origin_coordinates|start_date|  end_date|               lat|               lon|start_date|  end_date|wind_spd|
+---+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+--------------------+-------------------+----------+----------+------------------+------------------+----------+----------+--------+
|  1|2013|    1|  1|   533.0|           529|      4.0|   850.0|           830|    